# Chapter 4: Training Models

Bab ini membuka "kotak hitam" dari beberapa model Machine Learning. Kita akan melihat secara mendalam bagaimana model-model ini bekerja, yang akan membantu kita dalam memilih model yang tepat, algoritma pelatihan yang benar, dan set *hyperparameter* yang baik untuk sebuah tugas. Pemahaman ini juga krusial untuk melakukan *debugging* dan analisis kesalahan.

Fokus utama bab ini adalah pada **Regresi Linear**, menjelajahi dua cara yang sangat berbeda untuk melatihnya:
1. Menggunakan persamaan matematis langsung (*closed-form equation*) yang disebut **Normal Equation**.
2. Menggunakan pendekatan optimisasi iteratif yang disebut **Gradient Descent (GD)**, beserta variannya (Batch GD, Stochastic GD, dan Mini-batch GD).

Kita juga akan membahas **Regresi Polinomial** untuk data non-linear, cara mendeteksi *overfitting* menggunakan **learning curves**, dan berbagai teknik **regularisasi** (Ridge, Lasso, Elastic Net) untuk menguranginya. Terakhir, kita akan melihat model klasifikasi seperti **Regresi Logistik** dan **Regresi Softmax**.

## Regresi Linear

Model regresi linear membuat prediksi dengan menghitung jumlah berbobot dari fitur-fitur masukan, ditambah sebuah konstanta yang disebut *bias term* (atau *intercept term*).

**Persamaan Prediksi Regresi Linear**
$$ \hat{y} = \theta_0 + \theta_1 x_1 + \theta_2 x_2 + \dots + \theta_n x_n $$

Melatih model ini berarti mengatur parameternya ($\theta$) sehingga model paling sesuai dengan data pelatihan. Ukuran kinerja yang paling umum untuk regresi adalah **Mean Squared Error (MSE)**. Jadi, tujuan kita adalah menemukan nilai $\theta$ yang meminimalkan MSE.

### The Normal Equation

Normal Equation adalah formula matematis yang memberikan hasil secara langsung untuk menemukan nilai $\theta$ yang meminimalkan fungsi biaya.

**Normal Equation**
$$ \hat{\theta} = (X^T X)^{-1} X^T y $$

Ini adalah cara yang efisien untuk mendapatkan parameter optimal secara langsung, tetapi bisa menjadi sangat lambat jika jumlah fitur sangat besar.

In [ ]:
import numpy as np

# Membuat data linear acak
X = 2 * np.random.rand(100, 1)
y = 4 + 3 * X + np.random.randn(100, 1)

# Menambahkan x0 = 1 ke setiap instance
X_b = np.c_[np.ones((100, 1)), X]

# Menghitung theta menggunakan Normal Equation
theta_best = np.linalg.inv(X_b.T.dot(X_b)).dot(X_b.T).dot(y)

print(theta_best)

### Gradient Descent

Gradient Descent adalah algoritma optimisasi generik yang bekerja dengan menyesuaikan parameter secara iteratif untuk meminimalkan fungsi biaya. Ia mengukur gradien lokal dari fungsi kesalahan terhadap parameter $\theta$, dan bergerak ke arah gradien yang menurun. Ukuran langkahnya ditentukan oleh *learning rate*.

**Varian-varian Gradient Descent:**
* **Batch GD:** Menggunakan seluruh set pelatihan di setiap langkah. Sangat lambat pada dataset besar.
* **Stochastic GD (SGD):** Memilih satu instance acak di setiap langkah. Jauh lebih cepat tetapi kurang teratur; solusinya bagus tetapi tidak optimal.
* **Mini-batch GD:** Kompromi antara keduanya. Ia bekerja pada set kecil instance acak yang disebut *mini-batches*.

In [ ]:
# Implementasi Batch Gradient Descent sederhana
eta = 0.1  # learning rate
n_iterations = 1000
m = 100

theta = np.random.randn(2,1)  # inisialisasi acak

for iteration in range(n_iterations):
    gradients = 2/m * X_b.T.dot(X_b.dot(theta) - y)
    theta = theta - eta * gradients

print(theta)

## Regresi Polinomial

Bagaimana jika data Anda lebih kompleks dari sekadar garis lurus? Kita bisa menggunakan model linear untuk data non-linear dengan cara menambahkan pangkat dari setiap fitur sebagai fitur baru, lalu melatih model linear pada set fitur yang diperluas ini. Teknik ini disebut **Regresi Polinomial**.

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

# Membuat data non-linear
m = 100
X = 6 * np.random.rand(m, 1) - 3
y = 0.5 * X**2 + X + 2 + np.random.randn(m, 1)

# Menambahkan fitur polinomial
poly_features = PolynomialFeatures(degree=2, include_bias=False)
X_poly = poly_features.fit_transform(X)

# Melatih model regresi linear pada data yang diperluas
lin_reg = LinearRegression()
lin_reg.fit(X_poly, y)
print(lin_reg.intercept_, lin_reg.coef_)

## Learning Curves

*Learning curves* adalah plot kinerja model pada *training set* dan *validation set* sebagai fungsi dari ukuran *training set*. Kurva ini membantu kita mendiagnosis apakah model mengalami *overfitting* atau *underfitting*.

- Jika kedua kurva mencapai *plateau*, berdekatan, dan nilainya cukup tinggi, model kemungkinan besar **underfitting**.
- Jika ada celah besar antara kurva pelatihan dan kurva validasi, model kemungkinan besar **overfitting**.

## Model Linear yang Diregularisasi

Untuk mengurangi *overfitting*, kita bisa meregularisasi model, yaitu dengan membatasi bobotnya. Tiga jenis regularisasi yang umum adalah:

1.  **Ridge Regression (Regularisasi L2):** Menambahkan istilah regularisasi yang setara dengan setengah kuadrat dari norma L2 vektor bobot ke fungsi biaya. Ini mendorong bobot model sekecil mungkin.

2.  **Lasso Regression (Regularisasi L1):** Menggunakan norma L1 dari vektor bobot. Lasso cenderung menghilangkan bobot dari fitur yang paling tidak penting (mengaturnya menjadi nol), sehingga secara otomatis melakukan seleksi fitur.

3.  **Elastic Net:** Merupakan jalan tengah antara Ridge dan Lasso, dengan rasio campuran *r*. Ketika *r*=0, ia setara dengan Ridge, dan ketika *r*=1, ia setara dengan Lasso.

**Early stopping** adalah teknik regularisasi lain di mana pelatihan dihentikan segera setelah kesalahan validasi mencapai minimum.

## Regresi Logistik dan Softmax

* **Regresi Logistik:** Digunakan untuk memperkirakan probabilitas bahwa sebuah instance termasuk dalam kelas tertentu (klasifikasi biner). Ia menghitung jumlah berbobot dari fitur masukan dan mengeluarkan *logistic* dari hasil ini.

* **Regresi Softmax:** Merupakan generalisasi dari Regresi Logistik untuk mendukung banyak kelas secara langsung (klasifikasi multikelas). Ia menghitung skor untuk setiap kelas, lalu memperkirakan probabilitas setiap kelas dengan menerapkan fungsi *softmax*.